In [1]:
import pandas as pd
import numpy as np

ANALYSIS ON IDS

In [47]:
#import IDS data
IDS_data = pd.read_csv(r'C:\Users\user\Desktop\MOPA Reconciliation\Reconciliation week of 28 Sept\Season Clients Detailed_20200928-093633.csv', usecols=['GovLocationGrandParent','GovLocationParent','SiteName','LastName','FirstName','OAFID','NationalID','2021A_DAP kg','2021A_NPK 17 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg'])


In [86]:
MOPA = pd.read_excel(r'C:\Users\user\Desktop\MOPA Reconciliation\Reconciliation week of 28 Sept\MOPA data.xlsx')
MOPA['SectorID']=MOPA['District']+'_'+MOPA['Sector']

In [87]:
#Preliminary info on IDS data on subsidized inputs
IDS_data['SectorID'] = IDS_data['GovLocationGrandParent']+'_'+IDS_data['GovLocationParent']
SectorID = MOPA['SectorID'].unique().tolist()
#GovLocationGrandParent = 'Bugesera'
#GovLocationParent = 'Shyara'
SectorDf = IDS_data [IDS_data ['SectorID'].isin(SectorID)]
SectorClients = SectorDf.groupby('SectorID')['OAFID'].count()
SectorDf = SectorDf.rename(columns={'GovLocationGrandParent':'GovDistrict', 'GovLocationParent':'Sector'})

SubsInputsdf = SectorDf[['SectorID','GovDistrict','Sector','SiteName','LastName','FirstName','OAFID','NationalID','2021A_DAP kg','2021A_NPK 17 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg']]
SubsInputsdf['NationalID'] =SubsInputsdf['NationalID'].astype(str).replace('\.0', '', regex=True)

SubsInputsMelt = SubsInputsdf.melt(['SectorID','GovDistrict','Sector','SiteName','LastName','FirstName','OAFID','NationalID'],['2021A_DAP kg','2021A_NPK 17 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 403 kg','2021A_SC 637 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_WH 505 kg','2021A_WH 605 kg'], 'inputName','Quantity').fillna(0)

#Cleaning some IDS columns
SubsInputsMelt['NationalID'] =SubsInputsMelt['NationalID'].astype(str).replace('\.0', '', regex=True)

IDSInputsTxns = SubsInputsMelt[SubsInputsMelt['Quantity']>0]
IDSInputsTxns ['inputName'] = IDSInputsTxns ['inputName'].replace(['2021A_DAP kg','2021A_NPK 17 kg','2021A_UREA kg','2021A_WH 403 kg','2021A_SC 403 kg','2021A_Pan 53 kg','2021A_Pan 691 kg','2021A_SC 637 kg','2021A_WH 505 kg','2021A_WH 605 kg'],['DAP','NPK','UREA','WH403','SC403','PAN53','PAN691','SC637','WH505','WH605'])
IDSInputsTxns =IDSInputsTxns.rename(columns={'inputName':'Product'})
IDSInputsTxns['ProductID'] = IDSInputsTxns['NationalID'] + IDSInputsTxns['Product']

#Check if NID starts with 119
SubsInputsdf['NIDStartsWith119'] = SubsInputsdf['NationalID'].str.startswith('119')
NIDStartsWith119 = SubsInputsdf[SubsInputsdf['NIDStartsWith119']==False]
NIDStartsWith119_sector = NIDStartsWith119.groupby('SectorID')['OAFID'].count().fillna(0).reset_index()

#Check if NID contains either 700 for female and 800 for male
SubsInputsdf['FemaleCheck'] = SubsInputsdf['NationalID'].str.contains(pat = '700')
SubsInputsdf['maleCheck'] = SubsInputsdf['NationalID'].str.contains(pat = '800')
SubsInputsdf['FalseGender'] = SubsInputsdf['FemaleCheck']== SubsInputsdf['maleCheck']
FalseGender = SubsInputsdf[SubsInputsdf['FalseGender']==True]
FalseGender_sector = FalseGender.groupby('SectorID')['OAFID'].count().fillna(0).reset_index()

#Check if NID length is equal to 16
SubsInputsdf['NIDLength'] = SubsInputsdf['NationalID'].str.len()==16
NIDLength = SubsInputsdf[SubsInputsdf['NIDLength']==False]
NIDLength_sector = NIDLength.groupby('SectorID')['OAFID'].count().fillna(0).reset_index()

#FINAL NID FRAME
x = pd.merge(NIDStartsWith119_sector,FalseGender_sector, on='SectorID', how='outer').reset_index()
CheckOnNID = pd.merge(x,NIDLength_sector, on='SectorID',how='outer').fillna(0).reset_index()
CheckOnNID['#Wrong NID'] = CheckOnNID['OAFID_x']+CheckOnNID['OAFID_y']+CheckOnNID['OAFID']

#IDSClientsPerInput = IDSInputsTxns.groupby('Product')['OAFID'].count().reset_index()
#IDSQtyPerInput = IDSInputsTxns.groupby('Product')['Quantity'].sum().reset_index()
#SectorSummary = pd.merge(IDSClientsPerInput,IDSQtyPerInput,on='Product',how='outer').rename(columns={'OAFID':'#Clients'})
#SectorSummary 

#(1) FINDING THE TOTAL NUMBER OF CLIENTS IN EACH SECTOR ON IDS
TotalSectorClients = SectorDf.groupby('SectorID')['OAFID'].count().reset_index()

#(2)FINDING THE TOTAL NUMBER OF WRONG NID
ClientsWithWrongNID = CheckOnNID[['SectorID','#Wrong NID']]

#(3)FINDING THE TOTAL QUANTITY ON IDS IN EACH SECTOR
TotalKgsPerSector = pd.pivot_table(IDSInputsTxns, values='Quantity', index=['SectorID'],columns=['Product'], aggfunc='sum').reset_index().rename(columns={'DAP':'DAP I_kgs', 'NPK':'NPK I_kgs', 'SC403':'SC403 I_kgs' , 'UREA':'UREA I_kgs','WH403':'WH403 I_kgs','PAN53':'PAN53 I_kgs','PAN691':'PAN691 I_kgs','SC637':'SC637 I_kgs','WH505':'WH505 I_kgs'})

#(4)TOTAL NUMBER OF CLIENTS WHO TOOK SPECIFIC PRODUCTS
ClientsPerInputs = pd.pivot_table(IDSInputsTxns, values='OAFID', index=['SectorID'],columns=['Product'], aggfunc='count').reset_index().rename(columns={'DAP':'DAP I#', 'NPK':'NPK I#', 'SC403':'SC403 I#' , 'UREA':'UREA I#','WH403':'WH403 I#','PAN53':'PAN53 I#','PAN691':'PAN691 I#','SC637':'SC637 I#','WH505':'WH505 I#'})

#(5)MATCHING THE NUMBER OF CLIENTS AND TOTAL KGS TAKEN
IDS = pd.merge(pd.merge(ClientsPerInputs,TotalKgsPerSector, on='SectorID', how='outer' ),ClientsWithWrongNID, on='SectorID', how='outer').fillna(0)

In [88]:
IDS

,SectorID,DAP I#,NPK I#,PAN53 I#,PAN691 I#,SC403 I#,SC637 I#,UREA I#,WH403 I#,WH505 I#,DAP I_kgs,NPK I_kgs,PAN53 I_kgs,PAN691 I_kgs,SC403 I_kgs,SC637 I_kgs,UREA I_kgs,WH403 I_kgs,WH505 I_kgs,#Wrong NID
0,Bugesera_Gashora,1528.0,50.0,0.0,0.0,474.0,0.0,1343.0,1179.0,0.0,15200.0,900.0,0.0,0.0,1408.0,0.0,10622.5,3266.0,0.0,908.0
1,Bugesera_Ngeruka,995.0,21.0,0.0,0.0,936.0,0.0,798.0,0.0,0.0,20270.0,510.0,0.0,0.0,2918.0,0.0,8825.0,0.0,0.0,408.0
2,Bugesera_Shyara,2089.0,300.0,0.0,0.0,2060.0,0.0,1980.0,0.0,0.0,23105.0,4820.0,0.0,0.0,5157.0,0.0,17160.0,0.0,0.0,1079.0
3,Burera_Butaro,2536.0,1059.0,0.0,2814.0,0.0,0.0,1670.0,0.0,0.0,45215.0,53215.0,0.0,19658.0,0.0,0.0,18930.0,0.0,0.0,1366.0
4,Gatsibo_Gitoki,2081.0,308.0,1.0,0.0,2008.0,0.0,2125.0,0.0,0.0,30795.0,5380.0,10.0,0.0,7936.0,0.0,25485.0,0.0,0.0,1313.0
5,Gisagara_Gikonko,1150.0,117.0,316.0,0.0,777.0,0.0,1133.0,0.0,0.0,15975.0,2415.0,880.0,0.0,2754.0,0.0,9958.0,0.0,0.0,226.0
6,Huye_Tumba,705.0,169.0,9.0,0.0,663.0,0.0,766.0,0.0,0.0,8145.0,3110.0,27.0,0.0,1560.0,0.0,8702.5,0.0,0.0,113.0
7,Kamonyi_Rukoma,1769.0,513.0,0.0,0.0,1582.0,0.0,1757.0,0.0,0.0,15025.0,4735.0,0.0,0.0,3507.0,0.0,13608.0,0.0,0.0,256.0
8,Nyamasheke_Karambi,4089.0,772.0,1192.0,0.0,0.0,24.0,4014.0,0.0,1.0,68230.0,11780.0,2476.0,0.0,0.0,48.0,54688.5,0.0,2.0,470.0


ANALYSIS ON MOPA

In [89]:
#Cleaning MOPA data
#ColSplit = MOPA ['Requested Qty'].str.split(" ", n = 2, expand = True)
#MOPA['Requested_qty'] = ColSplit[0].astype(float)
MOPA['Paid_qty'] = MOPA['Paid Qty'].str.extract('(\d+)').astype(float)
MOPA['Product'] = MOPA['Product'].str.upper()
MOPA = MOPA.rename(columns={'National Id':'NationalID'})
MOPA['NationalID'] = MOPA['NationalID'].astype(str)
MOPA['ProductID'] = MOPA['NationalID'] + MOPA['Product']
MOPA['Product'] = MOPA['Product'].replace(['NPK 17:17:17'],['NPK'])
MOPA_dataCols = MOPA[['District','SectorID','Name','Surname','NationalID','Product','Paid_qty']]

In [93]:
MOPATotalKgsPerSector = pd.pivot_table(MOPA_dataCols, values='Paid_qty', index=['SectorID'],columns=['Product'], aggfunc='sum').reset_index().rename(columns={'DAP':'DAP M_kgs', 'NPK':'NPK M_kgs', 'SC403':'SC403 M_kgs' , 'UREA':'UREA M_kgs','WH403':'WH403 M_kgs','PAN691':'PAN691 M_kgs','SC637':'SC637 M_kgs','WH505':'WH505 M_kgs','PAN53':'PAN53 M_kgs','SC637':'SC637 M_kgs'}).fillna(0)

MOPAClientsPerInputs = pd.pivot_table(MOPA_dataCols, values='NationalID', index=['SectorID'],columns=['Product'], aggfunc='count').reset_index().rename(columns={'DAP':'DAP M#', 'NPK':'NPK M#', 'SC403':'SC403 M#' , 'UREA':'UREA M#','WH403':'WH403 M#','PAN691':'PAN691 M#','SC637':'SC637 M#','WH505':'WH505 M#','PAN53':'PAN53 M#','SC637':'SC637 M#'}).fillna(0)

MOPA_data = pd.merge(MOPATotalKgsPerSector,MOPAClientsPerInputs, on='SectorID', how='outer' )


In [95]:
MOPA_data

Product,SectorID,DAP M_kgs,NPK M_kgs,PAN53 M_kgs,PAN691 M_kgs,SC403 M_kgs,UREA M_kgs,WH403 M_kgs,WH505 M_kgs,DAP M#,NPK M#,PAN53 M#,PAN691 M#,SC403 M#,UREA M#,WH403 M#,WH505 M#
0,Bugesera_Gashora,6689.0,530.0,0.0,0.0,0.0,5994.0,46.0,0.0,854.0,22.0,0.0,0.0,0.0,807.0,20.0,0.0
1,Bugesera_Ngeruka,20342.0,570.0,0.0,0.0,88.0,8966.0,0.0,0.0,980.0,19.0,0.0,0.0,29.0,792.0,0.0,0.0
2,Bugesera_Shyara,21717.0,4640.0,0.0,0.0,84.0,16464.0,0.0,0.0,1936.0,283.0,0.0,0.0,26.0,1838.0,0.0,0.0
3,Burera_Butaro,38678.0,45529.0,0.0,113.0,0.0,16205.0,0.0,0.0,1982.0,739.0,0.0,19.0,0.0,1226.0,0.0,0.0
4,Gatsibo_Gitoki,37049.0,5430.0,0.0,0.0,150.0,31102.0,0.0,0.0,2331.0,312.0,0.0,0.0,28.0,2376.0,0.0,0.0
5,Gatsibo_Ngarama,61229.0,2325.0,351.0,0.0,0.0,58018.0,0.0,0.0,2426.0,117.0,42.0,0.0,0.0,2464.0,0.0,0.0
6,Gisagara_Gikonko,20423.0,2760.0,18.0,0.0,14.0,12843.0,0.0,0.0,1483.0,143.0,6.0,0.0,2.0,1458.0,0.0,0.0
7,Huye_Tumba,11375.0,3855.0,0.0,0.0,99.0,12063.0,0.0,0.0,991.0,220.0,0.0,0.0,35.0,1079.0,0.0,0.0
8,Kamonyi_Kayenzi,5295.0,3238.0,0.0,0.0,0.0,4425.0,0.0,60.0,630.0,236.0,0.0,0.0,0.0,570.0,0.0,28.0
9,Kamonyi_Rukoma,22062.0,5460.0,0.0,0.0,66.0,20033.0,0.0,0.0,2556.0,562.0,0.0,0.0,31.0,2542.0,0.0,0.0


In [96]:
#COMBINING IDS AND MOPA DATA
IDSvsMOPA = pd.merge(IDS,MOPA_data, on='SectorID',how='outer')
IDSvsMOPA['Diff in DAP#'] = IDSvsMOPA['DAP I#']-IDSvsMOPA['DAP M#']
IDSvsMOPA['Diff in NPK#'] = IDSvsMOPA['NPK I#']-IDSvsMOPA['NPK M#']
IDSvsMOPA['Diff in UREA#'] = IDSvsMOPA['UREA I#']-IDSvsMOPA['UREA M#']
#IDSvsMOPA['Diff in SC403#'] = IDSvsMOPA['SC403 I#']-IDSvsMOPA['SC403 M#']
IDSvsMOPA['Diff in WH403#'] = IDSvsMOPA['WH403 I#']-IDSvsMOPA['WH403 M#']
IDSvsMOPA['Diff in PAN53#'] = IDSvsMOPA['PAN53 I#'] - IDSvsMOPA['PAN53 M#']
IDSvsMOPA['Diff in PAN691#'] = IDSvsMOPA['PAN691 I#'] - IDSvsMOPA['PAN691 M#'] 
IDSvsMOPA['Diff in SC637#'] = IDSvsMOPA['SC637 I#']-IDSvsMOPA['SC637 M#']
IDSvsMOPA['Diff in WH505#'] = IDSvsMOPA['WH505 I#']-IDSvsMOPA['WH505 M#']

IDSvsMOPA['Diff in DAP Kgs'] = IDSvsMOPA['DAP I_kgs']-IDSvsMOPA['DAP M_kgs']
IDSvsMOPA['Diff in NPK Kgs'] = IDSvsMOPA['NPK I_kgs']-IDSvsMOPA['NPK M_kgs']
IDSvsMOPA['Diff in UREA Kgs'] = IDSvsMOPA['UREA I_kgs']-IDSvsMOPA['UREA M_kgs']
IDSvsMOPA['Diff in SC403 Kgs'] = IDSvsMOPA['SC403 I_kgs']-IDSvsMOPA['SC403 M_kgs']
IDSvsMOPA['Diff in WH403 Kgs'] = IDSvsMOPA['WH403 I_kgs']-IDSvsMOPA['WH403 M_kgs']
#IDSvsMOPA['Diff in SC403#'] = IDSvsMOPA['SC403 I#']-IDSvsMOPA['SC403 M#']
#IDSvsMOPA['Diff in PAN53 Kgs'] = IDSvsMOPA['PAN53 I_kgs'] - IDSvsMOPA['PAN53 M_kgs']
IDSvsMOPA['Diff in PAN691 Kgs'] = IDSvsMOPA['PAN691 I_kgs'] - IDSvsMOPA['PAN691 M_kgs'] 
IDSvsMOPA['Diff in SC637 Kgs'] = IDSvsMOPA['SC637 I_kgs']-IDSvsMOPA['SC637 M_kgs']
IDSvsMOPA['Diff in WH505 Kgs'] = IDSvsMOPA['WH505 I_kgs']-IDSvsMOPA['WH505 M_kgs']



IDSvsMOPA.to_excel('ArseneMOPA4.xlsx')


KeyError: 'SC637 M#'

In [61]:
MOPA_dataCols['Product'].unique()

array(['UREA', 'DAP', 'NPK', 'WH403', 'SC403'], dtype=object)

In [73]:
IDSvsMOPA.columns

Index(['SectorID', 'DAP I#', 'NPK I#', 'PAN53 I#', 'PAN691 I#', 'SC403 I#',
       'SC637 I#', 'UREA I#', 'WH403 I#', 'WH505 I#', 'DAP I_kgs', 'NPK I_kgs',
       'PAN53 I_kgs', 'PAN691 I_kgs', 'SC403 I_kgs', 'SC637 I_kgs',
       'UREA I_kgs', 'WH403 I_kgs', 'WH505 I_kgs', '#Wrong NID', 'DAP M_kgs',
       'NPK M_kgs', 'PAN53 M_kgs', 'PAN691 M_kgs', 'PRODUCT_x', 'SC403 M_kgs',
       'UREA M_kgs', 'WH403 M_kgs', 'WH505 M_kgs', 'DAP M#', 'NPK M#',
       'PAN53 M#', 'PAN691 M#', 'PRODUCT_y', 'SC403 M#', 'UREA M#', 'WH403 M#',
       'WH505 M#', 'Diff in DAP#', 'Diff in NPK#', 'Diff in UREA#',
       'Diff in WH403#', 'Diff in PAN53#', 'Diff in PAN691#'],
      dtype='object')

In [ ]:
#Preliminary info on IDS data on subsidized inputs

ClientsqtyPerInput =MOPA_dataCols.groupby('Product')['Requested_qty','Paid_qty'].sum().reset_index()
ClientsNumPerInput = MOPA_dataCols.groupby('Product')['NationalID'].count().reset_index()

#Number of clients who took each input in mopa and the total quantity taken per each input
SummaryMOPA = pd.merge(ClientsqtyPerInput,ClientsNumPerInput,on='Product', how='outer').rename(columns={'NationalID':'#Clients'})
SummaryMOPA['Product'] = SummaryMOPA['Product'].replace(['NPK 17:17:17'],['NPK'])
SummaryMOPA

In [ ]:
#Other data on IDS
TotalMOPAClients = MOPA['NationalID'].nunique()
#Transactions
TotalMOPATxn = MOPA['NationalID'].count()

MATCHING IDS TO MOPA 

In [ ]:
 MOPAvsIDS = pd.merge(SectorSummary,SummaryMOPA,on='Product',how='outer')
 MOPAvsIDS['Difference In #Clients'] = MOPAvsIDS['#Clients_x'] - MOPAvsIDS['#Clients_y']
 MOPAvsIDS['Difference In qty'] = MOPAvsIDS['Quantity'] - MOPAvsIDS['Paid_qty']
 MOPAvsIDS

CLIENT LEVEL CHECK

In [ ]:
IDSInputsTxns = IDSInputsTxns.rename(columns={'NationalID':'IDS_NationalID','Product':'IDS_Product','Quantity':'IDS_Quantity','GovLocationParent':'GovSector','GovLocationGrandParent':'GovDistrict'})
IDSInputsTxns['SiteID'] = IDSInputsTxns['GovSector']+IDSInputsTxns['SiteName']
MOPAInputsTxns = MOPA[['Name', 'Surname', 'NationalID','Product','Paid_qty','ProductID']]
MOPAInputsTxns = MOPAInputsTxns.rename(columns={'NationalID':'SNS_NationalID','Product':'SNS_Product','Paid_qty':'SNS_Quantity'})
IDSvsMOPA_Clients = pd.merge(IDSInputsTxns,MOPAInputsTxns,on='ProductID', how='outer')
IDSvsMOPA_Clients['Match'] = IDSvsMOPA_Clients['IDS_Quantity'] == IDSvsMOPA_Clients['SNS_Quantity']


IDSvsMOPA_Clients['IDS_NationalID'] = "'"+ IDSvsMOPA_Clients['IDS_NationalID']
IDSvsMOPA_Clients['SNS_NationalID'] = "'"+ IDSvsMOPA_Clients['SNS_NationalID']
path = r'C:\Users\user\Desktop\MOPA Reconciliation\Bugesera\MOPAvsIDS data'
os.chdir(path)
IDSvsMOPA_Clients.to_csv('Shyara.csv', index=False)

In [ ]:
#clients
TotalClientsOnMOPA_IDS = IDSvsMOPA_Clients['OAFID'].nunique()

#Transactions
MatchingTxn = IDSvsMOPA_Clients[IDSvsMOPA_Clients['Match']==True]['OAFID'].count()
NotMatchingTXn = IDSvsMOPA_Clients[IDSvsMOPA_Clients['Match']==False]['OAFID'].count()
OnIDSNotInMOPA = IDSvsMOPA_Clients[IDSvsMOPA_Clients['Paid_qty'].isna()]

In [ ]:
#Print clients
print('Total clients on IDS:',TotalIDSClients)
print('Total clients in MOPA:',TotalMOPAClients)
print('Total clients on IDS_MOPA df:',TotalClientsOnMOPA_IDS )


#Transactions
print('Total Txn on IDS:',IDSTotalTxn)
print('Total Txn in MOPA:',TotalMOPATxn)
print('Total Txn IDS match MOPA:',MatchingTxn)
print('Total Txn IDS dont match MOPA:',NotMatchingTXn)

In [ ]:
NIDCheck = IDSvsMOPA_Clients[['LastName','FirstName','OAFID','NationalID_x']]
NIDCheck['NIDStartsWith'] = NIDCheck['NationalID_x'].str.startswith('119')
NIDCheck['FemaleCheck'] = NIDCheck['NationalID_x'].str.contains(pat = '700')
NIDCheck['maleCheck'] = NIDCheck['NationalID_x'].str.contains(pat = '800')
NIDCheck['GenderCheck'] = NIDCheck['FemaleCheck']== NIDCheck['maleCheck']